In [1]:
#### 패키지 임포트
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import os
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
pd.set_option('display.max_columns', None)

In [2]:
#### 본문to형태소(morphs) 파일
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_3월_형태소분석.txt", encoding='UTF-8')
df['morphs']=df['morphs'].apply(lambda x : x.strip("'][").split("', '"))
print(df.shape, df.columns)
docs = df['morphs'].to_list()
print(len(docs))

(802683, 4) Index(['split_str', 'org_idx', 'morphs', 'pos_tag'], dtype='object')
802683


In [3]:
df

,split_str,org_idx,morphs,pos_tag
0,4월 전시회 추천 리움 미술관 순회전 인간 일곱 개의 질문 ft 전남도립미술관 인간...,0,"[4월, 전시회, 추천, 리움, 미술관, 순, 회전, 인간, 일곱, 개, 의, 질문...","[('4월', 'Number'), ('전시회', 'Noun'), ('추천', 'No..."
1,그때 너무 가고 싶었는데 어찌나 인기가 많던지 예약이 불가했어요,0,"[그때, 너무, 가고, 싶었는데, 어찌나, 인기, 가, 많던지, 예약, 이, 불가했어요]","[('그때', 'Noun'), ('너무', 'Adverb'), ('가다', 'Ver..."
2,전시 마지막날까지 전일매진 을 기록하는 뜨거운 반응을 보였는데요 그 전시가 이곳 전...,0,"[전시, 마지막, 날, 까지, 전일, 매진, 을, 기록, 하는, 뜨거운, 반응, 을...","[('전시', 'Noun'), ('마지막', 'Noun'), ('날', 'Noun'..."
3,그 전시를 오늘은 일을 핑계로 이렇게 편히 와서 봤네요,0,"[그, 전시, 를, 오늘, 은, 일, 을, 핑계, 로, 이렇게, 편히, 와서, 봤네요]","[('그', 'Noun'), ('전시', 'Noun'), ('를', 'Josa'),..."
4,전시명 인간 일곱 개의 질문 장소 전남도립미술관 주소 전남 광양시 광양읍 순광로 6...,0,"[전시, 명, 인간, 일곱, 개, 의, 질문, 장소, 전남, 도립, 미술관, 주소,...","[('전시', 'Noun'), ('명', 'Suffix'), ('인간', 'Noun..."
...,...,...,...,...
802678,엑실레스 요새 Forte di Exilles 카살레 몬페라토 또한 인상적인 방어 체...,16127,"[엑실레스, 요새, Forte, di, Exilles, 카, 살레, 몬페라토, 또한...","[('엑실레스', 'Noun'), ('요새', 'Noun'), ('Forte', '..."
802679,피네롤로 또한 요새화된 중요한 성채로 지리적 중요성으로 인하여 17세기 동안 프랑스...,16127,"[피, 네롤, 로, 또한, 요새화, 된, 중요한, 성채, 로, 지리, 적, 중요성,...","[('피', 'Noun'), ('네롤', 'Noun'), ('로', 'Josa'),..."
802680,베루아 사보이아 Verrua Savoia 의 요새는 복잡한 성벽을 갖추고 있어 17...,16127,"[베, 루아, 사보이아, Verrua, Savoia, 의, 요새, 는, 복잡한, 성...","[('베', 'Noun'), ('루아', 'Noun'), ('사보이아', 'Noun..."
802681,니차 또한 바다로 나가는 유일한 출구라는 중요성으로 인하여 요새화되었다,16127,"[니, 차, 또한, 바다로, 나가는, 유일한, 출구, 라는, 중요성, 으로, 인하여...","[('니', 'Modifier'), ('차', 'Noun'), ('또한', 'Nou..."


In [4]:
#### 각각의 본문이 str타입('나는 어제 밥을 먹었다')일 경우 list(['나', '는', '어제', '밥', '을' '먹다', '이다'])로 바꿔줌
docs2 = docs.copy()
# for i in tqdm(range(len(docs))):
#     docs2[i] = docs2[i].split()

In [5]:
#### 명사 빈도 상위 50개 단어 파일
topwords_df = pd.read_csv("../dataset/2022/본문_형태소분석/3월/나무문화_3월_형태소분석(명사).csv", encoding='UTF-8')
print(topwords_df.shape, topwords_df.columns)
topwords50 = topwords_df['명사'].to_list()[:50]
print(len(topwords50))
print(topwords50)

(142220, 2) Index(['명사', '명사빈도'], dtype='object')
50
['것', '수', '그', '이', '사람', '곳', '우리', '때', '등', '말', '내', '생각', '더', '나', '문화', '위', '나무', '길', '시간', '의', '때문', '하나님', '중', '일', '전', '제', '안', '사진', '위해', '하나', '마을', '대한', '아이', '개', '다른', '작품', '책', '집', '마음', '시작', '층', '공간', '속', '여행', '가지', '글', '가장', '날', '저', '한국']


In [6]:
#### 중복 단어 제거하여 리스트 생성
unique_lines = [list(set(line)) for line in tqdm(docs2)]

100%|██████████| 802683/802683 [00:05<00:00, 149189.34it/s]


In [7]:
#### 중복 제거 잘 됐나 개수로 확인
print(len(docs[0]))
print(len(unique_lines[0]))

37
29


In [8]:
#### 전체 단어쌍 빈도 dict 형식으로 생성
freq_count = {}   #동시출현 빈도가 저장될 dict
for words in tqdm(unique_lines):
    # 전체 단어에 대한 동시 출현 빈도
    for i, a in enumerate(words):
        for b in words[i+1:]:
            if a == b: continue
            elif a>b:
                freq_count[b, a] = freq_count.get((b, a),0) + 1
            else :
                freq_count[a, b] = freq_count.get((a, b),0) + 1

100%|██████████| 802683/802683 [11:01<00:00, 1213.45it/s] 


In [9]:
#### 딕셔너리를 데이터프레임에 담기
tt_freq_df=pd.DataFrame.from_dict(freq_count, orient='index')

In [10]:
#### 딕트to데이터프레임 저장
tt_freq_df.to_csv("../dataset/2022/단어동시출현행렬/3월/단어동시출현_딕셔너리_3월.csv", encoding='UTF-8')

In [11]:
#### dict형식을 컬럼명 지정해서 데이터프레임으로 만들기
list1 = [(tt_freq_df.index[i][0], tt_freq_df.index[i][1], tt_freq_df[0][i]) for i in tqdm(range(len(tt_freq_df)))]
freq_df2 = pd.DataFrame(list1, columns=["term1","term2","freq"])
print(freq_df2.shape)
freq_df2.head()

100%|██████████| 161400181/161400181 [19:39<00:00, 136788.04it/s] 


(161400181, 3)


,term1,term2,freq
0,개최,이,756
1,개최,작년,22
2,개최,기념,109
3,개최,관,68
4,4월,개최,159


In [12]:
#### 컬럼명 지정한 dict_to_데이터프레임 저장
freq_df2.to_csv("../dataset/2022/단어동시출현행렬/3월/단어동시출현_df_3월.csv", index=False, encoding='UTF-8')

In [13]:
#### 50위 이내에 속한 단어들로 이루어진 Dataframe을 추출
list2 =[(freq_df2.loc[i][0], freq_df2.loc[i][1], freq_df2.loc[i][2]) for i in tqdm(range(len(freq_df2))) if (freq_df2['term1'][i] in topwords50) &(freq_df2['term2'][i] in topwords50)]
freq_df3 = pd.DataFrame(list2, columns=["term1","term2","freq"])
freq_df3

100%|██████████| 161400181/161400181 [32:54<00:00, 81758.39it/s]


,term1,term2,freq
0,개,이,10277
1,의,이,180007
2,개,의,11597
3,곳,이,34797
4,날,이,11870
...,...,...,...
1220,글,하나님,108
1221,작품,하나님,43
1222,층,하나님,21
1223,여행,하나님,26


In [14]:
#### 상위50위 단어들로 이루어진 단어쌍 데이터프레임 저장
freq_df3.to_csv("../dataset/2022/단어동시출현행렬/3월/단어동시출현_df_상위50_3월.csv", index=False, encoding='UTF-8')

In [15]:
#### 공출현빈도(1-mode matrix) 만들기
word_co_matx = np.zeros((50,50))
mat_idx = topwords50 # 단어동시출현행렬이니까 인덱스와 칼럼을 동일하게 지정
mat_col = topwords50
word_co_matx = pd.DataFrame(word_co_matx, index=mat_idx, columns=mat_col)
print(word_co_matx.shape)


#### 단어쌍의 첫 번째 단어가 인덱스와 같다 and 두 번째 단어가 칼럼과 같다 → 단어쌍빈도를 해당 셀 값에 삽입
for n in tqdm(range(len(freq_df3))):
    for idx in mat_idx:
        if idx == freq_df3['term1'][n]:
            for col in mat_col:
                if col == freq_df3['term2'][n]:
                    word_co_matx.loc[idx, col] = freq_df3['freq'][n]
                    word_co_matx.loc[col, idx] = freq_df3['freq'][n]

(50, 50)


100%|██████████| 1225/1225 [00:00<00:00, 1476.59it/s]


In [16]:
#### 1-mode 매트릭스 확인하기
word_co_matx

,것,수,그,이,사람,곳,우리,때,등,말,내,생각,더,나,문화,위,나무,길,시간,의,때문,하나님,중,일,전,제,안,사진,위해,하나,마을,대한,아이,개,다른,작품,책,집,마음,시작,층,공간,속,여행,가지,글,가장,날,저,한국
것,0.0,25905.0,27398.0,97850.0,16282.0,8000.0,13754.0,13023.0,7267.0,12684.0,10766.0,11257.0,9350.0,12802.0,4668.0,7717.0,4529.0,3617.0,5113.0,74888.0,8585.0,7907.0,6573.0,8676.0,6239.0,4665.0,7164.0,2948.0,5954.0,5396.0,1787.0,5819.0,3353.0,3087.0,6300.0,3049.0,3532.0,3275.0,5480.0,3990.0,1316.0,2598.0,5636.0,2131.0,5060.0,1985.0,5172.0,3600.0,4551.0,2630.0
수,25905.0,0.0,14331.0,57856.0,9294.0,9295.0,7878.0,7593.0,8724.0,6457.0,6431.0,6246.0,6648.0,8388.0,5401.0,5133.0,4342.0,3715.0,5003.0,51775.0,5724.0,2703.0,4648.0,5450.0,4811.0,3993.0,4390.0,3217.0,3666.0,3285.0,2340.0,3853.0,3472.0,3052.0,3994.0,3024.0,2785.0,2473.0,3429.0,2402.0,2146.0,4708.0,4123.0,2314.0,3613.0,1582.0,3182.0,2299.0,2598.0,2196.0
그,27398.0,14331.0,0.0,55247.0,11435.0,7216.0,7866.0,9088.0,4911.0,7975.0,6632.0,5624.0,4912.0,7873.0,3048.0,5344.0,2927.0,2797.0,2940.0,53789.0,4929.0,5553.0,5896.0,5234.0,4421.0,3258.0,4899.0,1816.0,3887.0,3454.0,1405.0,3720.0,1831.0,2030.0,3557.0,2699.0,2214.0,2554.0,3405.0,2797.0,840.0,1673.0,4489.0,1215.0,3124.0,1368.0,2642.0,2806.0,2421.0,1675.0
이,97850.0,57856.0,55247.0,0.0,37273.0,34797.0,24862.0,31073.0,24596.0,24166.0,22450.0,21739.0,19751.0,27740.0,14032.0,16679.0,14379.0,12128.0,14872.0,180007.0,17295.0,11584.0,17311.0,17505.0,17410.0,13777.0,17177.0,11161.0,12366.0,11736.0,8220.0,11459.0,10107.0,10277.0,12520.0,9503.0,9724.0,10192.0,12082.0,10994.0,6004.0,10200.0,12331.0,6839.0,10582.0,6781.0,10853.0,11870.0,11954.0,7985.0
사람,16282.0,9294.0,11435.0,37273.0,0.0,3999.0,4546.0,5115.0,2473.0,5428.0,4158.0,4487.0,3369.0,5690.0,1835.0,2651.0,1735.0,1791.0,1918.0,27223.0,2976.0,3005.0,2743.0,3666.0,2209.0,1556.0,2997.0,1532.0,2397.0,1943.0,1228.0,1833.0,969.0,1129.0,3609.0,941.0,1360.0,1604.0,2463.0,1431.0,470.0,1029.0,2367.0,822.0,1929.0,902.0,1792.0,1527.0,1845.0,950.0
곳,8000.0,9295.0,7216.0,34797.0,3999.0,0.0,2319.0,3537.0,3336.0,1904.0,2661.0,2410.0,2586.0,3276.0,2916.0,2011.0,2365.0,3111.0,2414.0,20955.0,1671.0,474.0,2985.0,1744.0,2357.0,2229.0,2216.0,2531.0,1549.0,1615.0,2672.0,812.0,2117.0,1699.0,2116.0,775.0,746.0,1642.0,1440.0,1313.0,1161.0,2212.0,1562.0,2802.0,1064.0,738.0,2204.0,1349.0,2049.0,1082.0
우리,13754.0,7878.0,7866.0,24862.0,4546.0,2319.0,0.0,4279.0,1621.0,3584.0,2947.0,3168.0,2722.0,3513.0,2150.0,2491.0,1338.0,1409.0,1866.0,24195.0,2089.0,3966.0,1824.0,2787.0,1892.0,1129.0,2727.0,997.0,2253.0,1610.0,641.0,1694.0,1540.0,881.0,1675.0,691.0,1229.0,1775.0,2027.0,1143.0,410.0,778.0,2139.0,815.0,1618.0,708.0,1313.0,1290.0,1125.0,882.0
때,13023.0,7593.0,9088.0,31073.0,5115.0,3537.0,4279.0,0.0,2520.0,3914.0,3997.0,3475.0,2868.0,4860.0,1321.0,2161.0,1660.0,1571.0,2008.0,22519.0,2215.0,2094.0,2362.0,2980.0,2561.0,2018.0,2564.0,1319.0,1806.0,1646.0,795.0,1502.0,1406.0,1202.0,1858.0,865.0,1178.0,1621.0,1986.0,1838.0,496.0,806.0,1715.0,1165.0,1593.0,702.0,1527.0,1708.0,1803.0,936.0
등,7267.0,8724.0,4911.0,24596.0,2473.0,3336.0,1621.0,2520.0,0.0,1938.0,2272.0,1212.0,1500.0,3309.0,3750.0,2989.0,2362.0,1327.0,1442.0,25312.0,1333.0,214.0,2534.0,1951.0,2949.0,2738.0,1504.0,1141.0,1976.0,989.0,1305.0,1921.0,990.0,2214.0,1273.0,1676.0,992.0,1105.0,717.0,1277.0,1014.0,1816.0,1579.0,822.0,1359.0,631.0,1179.0,788.0,894.0,1857.0
말,12684.0,6457.0,7975.0,24166.0,5428.0,1904.0,3584.0,3914.0,1938.0,0.0,3848.0,2655.0,2247.0,4201.0,1231.0,1909.0,1190.0,955.0,1199.0,20146.0,1783.0,1792.0,1723.0,2552.0,1824.0,1496.0,2236.0,657.0,1454.0,1460.0,502.0,1353.0,1039.0,874.0,1609.0,637.0,1039.0,962.0,1583.0,1093.0,253.0,496.0,1582.0,460.0,1415.0,841.0,1026.0,1024.0,1286.0,712.0


In [17]:
#### 공출현빈도 데이터프레임 저장하기~~
#### ※ 인덱스가 저장돼야 하므로 index 파라미터 지정하지 말 것.
word_co_matx.to_csv("../dataset/2022/단어동시출현행렬/3월/공출현빈도_df_3월.csv", encoding='UTF-8')